## 事前準備

### moduleフォルダのパスを通す

In [1]:
from pathlib import Path
import sys
import os
PROJECT_FOLDER = str(Path(os.getcwd()).parents[1])
ORIGINAL_MODULES = PROJECT_FOLDER + '/modules'
sys.path.append(ORIGINAL_MODULES)

### 環境変数.envを読み込み

In [2]:
from pathlib import Path
from dotenv import load_dotenv
import os
env_path = f'{ORIGINAL_MODULES}\.env'
load_dotenv(dotenv_path=env_path)

True

### モジュールのインポート

In [3]:
# 自作モジュール
from acquisition.jquants_api_operations import run_jquants_api_operations
# 基本モジュール
from datetime import datetime
import pandas as pd
import numpy as np
# クラスタリングで使用
import umap
from pyclustering.cluster.xmeans import xmeans
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist

## 実行

### パラメータ

In [4]:
SECTOR_REDIM_FOLDER = f'{PROJECT_FOLDER}\sector_redefinitions'

start_date = datetime(2014, 1, 1)
end_date = datetime(2021, 12, 31)
INPUT_CSV = f'{SECTOR_REDIM_FOLDER}\DetermineSector_NewTickers2024.csv'
OUTPUT_CSV = f'{SECTOR_REDIM_FOLDER}DetermineSector_NewTickers2024.csv'

### 銘柄情報の取得
* stock_lists: 2014年10月からの銘柄一覧
* history_list: 銘柄ごとのScaleCategoryの遍歴

In [5]:
# 対象銘柄を絞り込んでデータを読み込み
tickers_df = pd.read_csv(INPUT_CSV, dtype='str')
tickers_code_list = tickers_df['Code'].to_list()
universe_filter = f"Code in {tickers_code_list}" #現行のTOPIX500
list_df, fin_df, price_df = run_jquants_api_operations(filter=universe_filter)
stock_dfs = {'stock_list': list_df, 'stock_fin': fin_df, 'stock_price': price_df}
stock_dfs['stock_list']

stock_dfデータ群の読み込みを開始します。
銘柄一覧の読み込みが完了しました。
価格情報の読み込みが完了しました。
財務情報の読み込みが完了しました。
stock_dfデータ群の読み込みが完了しました。
----------------------------------------------


,Code,CompanyName,MarketCodeName,Sector33CodeName,Sector17CodeName,ScaleCategory,Listing
19,1332,ニッスイ,プライム,水産・農林業,食品,TOPIX Mid400,1
20,1333,マルハニチロ,プライム,水産・農林業,食品,TOPIX Mid400,1
58,1414,ショーボンドホールディングス,プライム,建設業,建設・資材,TOPIX Mid400,1
59,1417,ミライト・ワン,プライム,建設業,建設・資材,TOPIX Mid400,1
173,1605,ＩＮＰＥＸ,プライム,鉱業,エネルギー資源,TOPIX Mid400,1
...,...,...,...,...,...,...,...
4437,2427,アウトソーシング,プライム,サービス業,情報通信・サービスその他,-,0
4442,2412,ベネフィット・ワン,プライム,サービス業,情報通信・サービスその他,-,0
4443,9783,ベネッセホールディングス,プライム,サービス業,情報通信・サービスその他,-,0
5539,4739,伊藤忠テクノソリューションズ,プライム,情報･通信業,情報通信・サービスその他,-,0


In [6]:
new_sector_price_df, order_price_df = \
    calculation.SectorIndexCalculator.calc_new_sector_price(stock_dfs, INPUT_CSV, None)
sectors = new_sector_price_df.index.get_level_values('Sector').unique()
raw_target_df = calculation.TargetCalculator.daytime_return(new_sector_price_df)

セクターのインデックス値の算出が完了しました。
特徴量（日中リターン）の算出が完了しました。


In [15]:
ticker = 'NSD'

corr_df = raw_target_df.unstack().corr().droplevel(0, axis=1)
corr_df.iloc[:, corr_df.index.get_level_values(1) == ticker].sort_values(ticker, ascending=False)

Sector                  NSD
       Sector              
Target NSD         1.000000
       インフラ        0.542634
       医療機器等       0.495824
       ITインフラ      0.487170
       素材          0.481660
       その他工業       0.480792
       文具・物流       0.456252
       小売・サービス     0.455624
       メディア        0.447899
       住宅          0.447681
       光学          0.441038
       食品          0.438714
       中堅医薬品       0.432773
       大手医薬品       0.423443
       富士ソフト       0.422694
       ビジネス        0.422349
       電気機器        0.409231
       化粧・衛生       0.406956
       私鉄          0.396354
       医薬品卸        0.395616
       化学          0.395028
       日鉄ソリューションズ  0.391924
       トヨタ他        0.387284
       電気機器2       0.383848
       証券・金融       0.378043
       医療機器販売      0.375710
       ガラス・機械      0.375474
       自動車         0.374997
       ゼネコン        0.361244
       ロボ・産機       0.359490
       エンタメ        0.359199
       OA機器        0.355258
       ドラッグストア     0.350703
       半導体         0.345882
       ニチアス        0.345041
       電子部品        0.343177
       スーパー・コンビニ   0.341331
       旅客          0.338035
       商社          0.337153
       外食          0.336994
       タイヤ         0.333945
       ディベロッパー     0.321951
       地銀          0.307649
       非鉄金属        0.301482
       ガス          0.290656
       百貨店         0.289294
       都銀・保険       0.279709
       ビックカメラ      0.265733
       電力          0.262763
       鉄鋼          0.259774
       MARUWA      0.257083
       重工業         0.251242
       JVCケンウッド    0.246975
       石油・資源       0.244749
       ビジョナル       0.210118
       海運          0.199899
       楽天銀行        0.156459

In [8]:
target_df

Target
Date       Sector            
2014-01-06 ITインフラ    0.001003
           JVCケンウッド -0.012326
           MARUWA   -0.000651
           NSD       0.033605
           OA機器      0.007149
...                       ...
2024-11-07 電気機器     -0.001623
           電気機器2    -0.021228
           非鉄金属      0.002468
           食品        0.000429
           高砂熱学工業    0.023661

[161711 rows x 1 columns]

### クラスタリングの関数

In [21]:
# UMAPを適用する関数
def apply_UMAP(df:pd.DataFrame, n_components:int=15, n_neighbors:int=5, min_dist:float=0.1):
    UMAP_model = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist)
    UMAP_result = UMAP_model.fit_transform(df)
    reduced_df = pd.DataFrame(UMAP_result, index=df.index, columns=['Feature '+ str(i) for i in range(0, n_components)])
    reduced_df = reduced_df.sort_index(ascending=True)
    return reduced_df

# xmeansのクラスタ中心を算出するサブ関数
def _calculate_xmeans_cluster_center(df, kmax):
    xm_c = kmeans_plusplus_initializer(data=df, amount_centers=2).initialize()
    xm_i = xmeans(data=df, initial_centers=xm_c, kmax=kmax, tolerrance=0.005, ccore=True)
    xm_i.process()

    for cluster_num, cluster in enumerate(xm_i._xmeans__clusters):
        for index_num in cluster:
            df.loc[df.index[index_num]] = np.array(xm_i._xmeans__centers, dtype=np.float32)[cluster_num, :]

    df = df.reset_index()
    return df

# データフレームにxmeansを適用
def apply_xmeans(df:pd.DataFrame, n_iteration:int=30, kmax=10):
    dfs_xmeans = []
    for i in range(0, n_iteration):
        df_xmeans_cluster_center = _calculate_xmeans_cluster_center(df, kmax=kmax)
        dfs_xmeans.append(df_xmeans_cluster_center)
    dfs_xmeans = pd.concat(dfs_xmeans, axis=0)
    dfs_xmeans = dfs_xmeans.reset_index().drop('index', axis=1)
    df = dfs_xmeans.groupby('Code').mean()
    return df

# 銘柄間のユークリッド距離を算出するサブ関数
def _calculate_euclidean(df:pd.DataFrame):
    np_euclidean = cdist(df.values, df.values, metric='euclidean')
    df_euclidean = pd.DataFrame(np_euclidean, index=df.index, columns=df.index)
    return df_euclidean

# ユークリッド距離からクラスタを決定するサブ関数
def _determine_cluster(df:pd.DataFrame, theshold_column:str, cluster_theshold:float):
    df_unstacked = pd.DataFrame(df.unstack(), columns=[theshold_column])
    in_theshold = df_unstacked[theshold_column]<=cluster_theshold
    df_in_theshold = df_unstacked[in_theshold].sort_values(theshold_column, ascending=True)
    df_in_theshold['Cluster'] = np.nan

    cluster_num = 0
    for i in df_in_theshold.index.get_level_values(0).unique():

        clustering_completed = df_in_theshold['Cluster'].isnull().any() == False
        if clustering_completed:
            break
        cluster_is_undetermined = \
          df_in_theshold.loc[df_in_theshold.index.get_level_values(0)==i, 'Cluster'].isnull().all()
        if cluster_is_undetermined:
            df_in_theshold.loc[df_in_theshold.index.get_level_values(0)==i, 'Cluster'] = cluster_num
            df_in_theshold.loc[df_in_theshold.index.get_level_values(1)==i, 'Cluster'] = cluster_num
            cluster_num += 1
        else:
            df_in_theshold.loc[df_in_theshold.index.get_level_values(0)==i, 'Cluster'] = \
                int(df_in_theshold.loc[df_in_theshold.index.get_level_values(0)==i, 'Cluster'].mean())
            df_in_theshold.loc[df_in_theshold.index.get_level_values(1)==i, 'Cluster'] = \
                int(df_in_theshold.loc[df_in_theshold.index.get_level_values(0)==i, 'Cluster'].mean())

    df_in_theshold = df_in_theshold[['Cluster']].astype(int).reset_index(level=0, drop=True)
    df_in_theshold = df_in_theshold.loc[~df_in_theshold.index.duplicated(keep='first')]

    df_clustered = pd.merge(
                            df_in_theshold, tickers_df[['Code', 'CompanyName', 'Sector33CodeName', 'ScaleCategory']],
                            how='left', on='Code',
                            ).sort_values(by=['Cluster', 'Code'], ascending=True).set_index('Code', drop=True)
    return df_clustered

# ユークリッド距離からクラスタを決定する関数
def determine_cluster_from_euclidean(df:pd.DataFrame, theshold_column='Distance', cluster_theshold=0.03):
    df_euclidean = _calculate_euclidean(df=df)
    df_clustered = _determine_cluster(df=df_euclidean, theshold_column=theshold_column, cluster_theshold=cluster_theshold)
    return df_clustered

### クラスタリング

In [10]:
#UMAP設定
start_date = datetime(2014,1,1) #UMAPの範囲（自）
end_date = datetime(2021,12,31) #UMAPの範囲（至）
n_components = 100 #何次元まで削減するか

#xmeans設定
n_iteration = 30
kmax = 2

In [11]:
# UMAPの適用
umaped_df = apply_UMAP(target_df.unstack(0),
                       n_components = 20,
                       n_neighbors = 5,
                       min_dist = 0.1)
umaped_df.index.name = 'Code'
umaped_df.head(2)

,Feature 0,Feature 1,Feature 2,Feature 3,Feature 4,Feature 5,Feature 6,Feature 7,Feature 8,Feature 9,Feature 10,Feature 11,Feature 12,Feature 13,Feature 14,Feature 15,Feature 16,Feature 17,Feature 18,Feature 19
Code,,,,,,,,,,,,,,,,,,,,
ITインフラ,7.324204,2.982087,3.925066,6.582467,6.434377,6.848620,4.793181,7.619975,3.581341,3.434576,6.505476,3.960529,7.034010,4.005229,6.134737,4.905088,1.364661,4.324255,4.652064,3.854821
JVCケンウッド,6.916071,3.436974,4.041061,5.964068,6.646266,7.546284,4.811968,8.314470,3.172635,4.246459,7.338310,2.925046,6.656809,3.003574,7.312142,5.298223,2.642885,3.932862,7.157615,5.613846


In [24]:
import warnings
np.warnings = warnings #np.warningsが存在しないエラーの対策
dfs_clustered = []
for n in range(100):
    print(n)
    # xmeansを適用してクラスタ中心を求める
    df_xmeans = apply_xmeans(df=umaped_df, n_iteration=30, kmax=20)
    # 求めたクラスタ中心を利用してクラスタリング
    xm_c = kmeans_plusplus_initializer(data=df_xmeans, amount_centers=2).initialize()
    xm_i = xmeans(data=df_xmeans, initial_centers=xm_c, kmax=2, tolerrance=0.001, ccore=True)
    xm_i.process()

    #初回のクラスタリング
    cluster_nums = []
    tickers = []
    for cluster_num, ticker_num in enumerate(xm_i._xmeans__clusters):
        cluster_nums += [cluster_num for _ in range(len(ticker_num))]
        tickers += df_xmeans.index[ticker_num].values.tolist()

    df_clustered = pd.DataFrame(tickers, columns=['Code'])
    df_clustered = pd.merge(df_clustered, 
                tickers_df[['Code', 'CompanyName', 'Sector33CodeName']].drop_duplicates(subset='Code', keep='last'),
                how='left',
                on='Code')
    df_clustered['Cluster'] = cluster_nums

    #display(df_clustered)
    '''
    for cluster in df_clustered['Cluster'].unique():
        print(f'Cluster{cluster}: {len(df_clustered[df_clustered["Cluster"]==cluster])} tickers')
    '''

    last_fixed_list = []

    for j in range(0, 10000):
        #print('----------------------------------')
        #print(f'Roop {j}')
        df_clustered = df_clustered.rename(columns={'Cluster':f'Cluster_{j}'})
        df_clustered_list = []
        fixed_cluster_num_list = []
        n = 0

        for i in range(df_clustered[f'Cluster_{j}'].max() + 1):
            #print(i)
            df = df_clustered[df_clustered[f'Cluster_{j}']==i].copy()
            df['Code'] = df['Code'].astype(str)
            if i in last_fixed_list:
                #print('Already Fixed')
                df['Cluster'] = n
                fixed_cluster_num_list.append(n)
                n += 1
                df_clustered_list.append(df)
                #display(df)
            else:
                code_list = df['Code'].unique().tolist()
                #stock_price_extracted = target_df[target_df.index.get_level_values('Sector').isin(code_list)]
                stock_price_reduced = target_df[target_df.index.get_level_values('Sector').isin(code_list)]
                try:
                    '''
                    stock_price_reduced = \
                        TargetCalculator._get_PCAresiduals(stock_price_extracted, 
                                                            reduce_components=1,
                                                            train_start_day=datetime(2013, 4, 5), 
                                                            train_end_day=datetime(2021, 12, 31))
                    '''
                    stock_price_reduced.index.names = ['Date', 'Code']
                    reduced_df = apply_UMAP(stock_price_reduced.unstack(0),
                        n_components = 15,
                        n_neighbors = 5,
                        min_dist = 0.1)
                    reduced_df = reduced_df.sort_index(ascending=True)
                    

                    df_xmeans = apply_xmeans(df=reduced_df, n_iteration=30, kmax=2)
                    # 求めたクラスタ中心を利用してクラスタリング
                    xm_c = kmeans_plusplus_initializer(data=df_xmeans, amount_centers=2).initialize()
                    xm_i = xmeans(data=df_xmeans, initial_centers=xm_c, kmax=2, tolerrance=0.0001, ccore=True)
                    xm_i.process()

                    cluster_nums = []
                    tickers = []
                    for cluster_num, ticker_num in enumerate(xm_i._xmeans__clusters):
                        cluster_nums += [cluster_num for _ in range(len(ticker_num))]
                        tickers += df_xmeans.index[ticker_num].values.tolist()

                    df_clustered_new = pd.DataFrame(tickers, columns=['Code'])
                    df_clustered_new = pd.merge(df_clustered_new, 
                                tickers_df[['Code', 'CompanyName', 'Sector33CodeName']].drop_duplicates(subset='Code', keep='last'),
                                how='left',
                                on='Code')
                    df_clustered_new['Cluster'] = cluster_nums

                    df_clustered_new['Cluster'] = df_clustered_new['Cluster'].replace({0:n, 1:n+1})
                    n += 2

                    df = pd.merge(df, df_clustered_new[['Code', 'Cluster']], how='inner', on='Code')
                    df = df.sort_values(by='Cluster', ascending=True)
                    df_clustered_list.append(df)
                    #display(df)
                except Exception as e:
                    print(e)
                    df['Cluster'] = n
                    fixed_cluster_num_list.append(n)
                    n += 1
                    df_clustered_list.append(df)
                    #display(df)
        df_clustered = pd.concat(df_clustered_list, axis=0)
        
        #for cluster in df_clustered['Cluster'].unique():
            #print(f'Cluster{cluster}: {len(df_clustered[df_clustered["Cluster"]==cluster])} tickers')
        
            
        last_fixed_list = fixed_cluster_num_list
        if all(df_clustered['Cluster'] == df_clustered[f'Cluster_{j}']):
            df_clustered = df_clustered[['Code', 'CompanyName', 'Cluster']]
            dfs_clustered.append(df_clustered)
            break
            
print('Completed!')

0


c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenva

Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
1


c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(


Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.


...

c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenva

Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
98


c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matri

Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
99
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.
Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linal

c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
c:\Users\ryosh\enrich_me\.venv\Lib\site-packages\umap\spectral.py:519: RuntimeWarning: k >= N for N * N square matri

In [470]:
dfs = pd.DataFrame(index=[dfs_clustered[0]['Code'], dfs_clustered[0]['CompanyName']])
for i, df_clustered in enumerate(dfs_clustered):
    df_clustered = df_clustered.set_index(['Code', 'CompanyName'], drop=True)
    df_clustered.columns = [f'Cluster{i}']
    dfs = pd.merge(dfs, df_clustered, how='inner', left_index=True, right_index=True)

dfs.shape

(317, 100)

In [471]:
sum_df = pd.DataFrame(index=dfs.index, columns=dfs.index)

for index1, row1 in dfs.iterrows():
    for index2, row2 in dfs.iterrows():
        sum_df.loc[index1, index2] = (row1 == row2).sum()

sum_df

,Code,オカムラ,ＥＩＺＯ,極東開発工業,オイレス工業,日東工業,マックス,新晃工業,大阪ソーダ,アパレル1,専門商社2,...,ネクステージ,コメダホールディングス,外食1,エンタメ,王将フードサービス,地方私鉄,九州旅客鉄道,外食2・学習塾,鎌倉新書,西武ホールディングス
,CompanyName,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Code,CompanyName,,,,,,,,,,,,,,,,,,,,,
オカムラ,NaN,100,11,48,74,36,45,54,29,41,58,...,0,0,0,0,0,0,0,0,0,0
ＥＩＺＯ,NaN,11,100,22,8,21,25,17,36,21,11,...,0,0,0,0,0,0,0,0,0,0
極東開発工業,NaN,48,22,100,54,39,31,35,26,32,51,...,0,0,0,0,0,0,0,0,0,0
オイレス工業,NaN,74,8,54,100,35,39,48,25,38,74,...,0,0,0,0,0,0,0,0,0,0
日東工業,NaN,36,21,39,35,100,32,24,23,32,39,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
地方私鉄,NaN,0,0,0,0,0,0,0,0,0,0,...,59,61,63,96,58,100,70,61,29,64
九州旅客鉄道,NaN,0,0,0,0,0,0,0,0,0,0,...,30,32,34,66,29,70,100,32,13,90


In [472]:
#sum_df.to_csv('TOPIX1000_ClusterProbability.csv', encoding='utf-8')

In [473]:
dist_df = sum_df
kmeans = KMeans(n_clusters=150, random_state=0)
dist_df['Cluster'] = kmeans.fit_predict(dist_df)
distlist = []
for index, row in dist_df.iterrows():
    distlist.append(dist_df.loc[dist_df['Cluster']==row['Cluster'].values[0], index].mean())

dist_df['MeanDistIntraCluster'] = distlist

dist_df= dist_df[['Cluster', 'MeanDistIntraCluster']]
dist_df = dist_df.droplevel(1, axis=1)
dist_df.columns.name = ''

dist_df.head()

,,Cluster,MeanDistIntraCluster
Code,CompanyName,,
オカムラ,NaN,100,72.8
ＥＩＺＯ,NaN,106,75.5
極東開発工業,NaN,45,74.5
オイレス工業,NaN,100,79.8
日東工業,NaN,57,100.0


In [474]:
dist_df.to_csv('TOPIX1000_DeterminedClusterPartial.csv', encoding='utf-8')

In [475]:

# 1回目の場合のみこのセルをオン
code_and_name = stock_dfs_dict['stock_list'].loc[stock_dfs_dict['stock_list']['Code'].isin(codes), ['Code', 'CompanyName']]
df = pd.read_csv(input_csv, encoding='utf-8')
df = df[['Code', 'ClusterName']]
code_and_name['Code'] = code_and_name['Code'].astype(str)
df['Code'] = df['Code'].astype(str)
code_and_name = pd.merge(code_and_name, df, how='outer', on='Code')
code_and_name.loc[code_and_name['ClusterName'].isna(), 'ClusterName'] = code_and_name['CompanyName']
a = dist_df.reset_index(drop=False)[['Code', 'Cluster', 'MeanDistIntraCluster']]
a = a.rename(columns={'Code':'ClusterName'})
df = pd.merge(code_and_name, a, how='left', on='ClusterName')
df.to_csv(output_csv, encoding='utf-8')


In [476]:
'''
dist_df = dist_df.reset_index()[['Code', 'Cluster', 'MeanDistIntraCluster']]
dist_df = dist_df.rename(columns={'Code': 'ClusterName'})
df = pd.read_csv(input_csv, encoding='utf-8')
df = pd.merge(df,dist_df, how='left', on='ClusterName')
df.to_csv(output_csv, encoding='utf-8')
'''

"\ndist_df = dist_df.reset_index()[['Code', 'Cluster', 'MeanDistIntraCluster']]\ndist_df = dist_df.rename(columns={'Code': 'ClusterName'})\ndf = pd.read_csv(input_csv, encoding='utf-8')\ndf = pd.merge(df,dist_df, how='left', on='ClusterName')\ndf.to_csv(output_csv, encoding='utf-8')\n"

In [477]:

x = 'コメダホールディングス'
a = sum_df.droplevel(-1, axis=0).droplevel(-1, axis=1)
a[[x]].sort_values(x, ascending=False).head(10)


Code,コメダホールディングス
Code,
フジオフードグループ本社,100
外食2・学習塾,100
コメダホールディングス,100
ＦＯＯＤ ＆ ＬＩＦＥ ＣＯＭＰＡＮＩＥＳ,99
外食1,98
すかいらーくホールディングス,98
ネクステージ,96
テクノプロ・ホールディングス,92
王将フードサービス,92


In [478]:
df

,Code,CompanyName,ClusterName,Cluster,MeanDistIntraCluster
0,1332,ニッスイ,食品1,20.0,88.750000
1,1333,マルハニチロ,食品1,20.0,88.750000
2,1377,サカタのタネ,食品1,20.0,88.750000
3,1379,ホクト,食品運搬・販売,30.0,84.000000
4,1414,ショーボンドホールディングス,土木・道路,28.0,79.500000
...,...,...,...,...,...
1129,9984,ソフトバンクグループ,樹脂等,2.0,98.214286
1130,9987,スズケン,医薬品卸,107.0,91.000000
1131,9989,サンドラッグ,ドラッグストア,67.0,99.142857
1132,9990,サックスバー ホールディングス,小売2,0.0,100.000000


In [479]:
'''
df2 = pd.read_csv('TOPIX1000_DeterminedCluster2.csv', encoding='utf-8')
df2 = df2[['Code', 'Cluster', 'MeanDistIntraCluster']]
df2.columns = ['ClusterName', 'Cluster', 'MeanDistIntraCluster']
df = pd.merge(df, df2, how='inner', on='ClusterName')
df = df.rename(columns={'ClusterName':'ClusterName1'})
df.to_csv('TOPIX1000_DeterminedCluster2.csv', encoding='utf-8')
'''

"\ndf2 = pd.read_csv('TOPIX1000_DeterminedCluster2.csv', encoding='utf-8')\ndf2 = df2[['Code', 'Cluster', 'MeanDistIntraCluster']]\ndf2.columns = ['ClusterName', 'Cluster', 'MeanDistIntraCluster']\ndf = pd.merge(df, df2, how='inner', on='ClusterName')\ndf = df.rename(columns={'ClusterName':'ClusterName1'})\ndf.to_csv('TOPIX1000_DeterminedCluster2.csv', encoding='utf-8')\n"